In [1]:
import yfinance as yf, pandas as pd, plotly.express as px

# get stock info
BTCCAD = yf.Ticker("BTC-CAD")
print(BTCCAD.info)

# get historical market data
CADBTC_df = BTCCAD.history(period="1y", interval='1h')
CADBTC_df = CADBTC_df.drop(columns=['Dividends','Stock Splits','Volume'])

#display(CADBTC_df)

{'name': 'Bitcoin', 'startDate': 1278979200, 'description': 'Bitcoin (BTC) is a cryptocurrency launched in 2010. Users are able to generate BTC through the process of mining. Bitcoin has a current supply of 19,727,703. The last known price of Bitcoin is 63,764.3968645 USD and is down -1.73 over the last 24 hours. It is currently trading on 11607 active market(s) with $28,028,532,972.56 traded over the last 24 hours. More information can be found at https://bitcoin.org/.', 'maxAge': 86400, 'priceHint': 2, 'previousClose': 87666.46, 'open': 87666.46, 'dayLow': 86812.24, 'dayHigh': 92492.36, 'regularMarketPreviousClose': 87666.46, 'regularMarketOpen': 87666.46, 'regularMarketDayLow': 86812.24, 'regularMarketDayHigh': 92492.36, 'volume': 49282191360, 'regularMarketVolume': 49282191360, 'averageVolume': 37211473091, 'averageVolume10days': 39212486913, 'averageDailyVolume10Day': 39212486913, 'marketCap': 1816125767680, 'fiftyTwoWeekLow': 33856.79, 'fiftyTwoWeekHigh': 99397.766, 'fiftyDayAver

In [2]:
#from datetime import datetime
from dateutil import tz

def convert_UTC_to_NY(UTCdatetime):
    # METHOD 1: Hardcode zones:
    from_zone = tz.gettz('UTC')
    to_zone = tz.gettz('America/New_York')

    # Tell the datetime object that it's in UTC time zone since datetime objects are 'naive' by default
    utc = UTCdatetime.replace(tzinfo=from_zone)

    # Convert time zone
    central_time = utc.astimezone(to_zone)
    return central_time

CADBTC_df.index = CADBTC_df.index.map(convert_UTC_to_NY)

#CADBTC_df

In [3]:
#filter table for 4pm on fridays
CADBTC_df['Day_of_week'] = CADBTC_df.index.strftime("%A")
CADBTC_df = CADBTC_df[CADBTC_df['Day_of_week'] == "Friday"]
CADBTC_df = CADBTC_df.at_time('16:00')
CADBTC_df = CADBTC_df.drop(columns=["Day_of_week","Open","Low","High"])

#CADBTC_df

In [4]:
### Convert to Sats nominator ###
#define function
def SatsConv(x):
  SatsPerCAD = 1/(x/100000000)
  return SatsPerCAD

#convert prices to sats
SATSCAD_df = CADBTC_df.map(SatsConv)

#view
#display(SATSCAD_df)

In [5]:
# Area chart

area_chart = px.area(SATSCAD_df['Close'], title = 'Satoshis per Canadian Dollar', color_discrete_map={"Close": "DarkOrange"})

area_chart.update_xaxes(title_text = 'Date')
area_chart.update_yaxes(title_text = 'Satoshis per CAD$')
area_chart.update_layout(showlegend = False)

area_chart.show()

In [6]:
area_chart.write_image("images/SatsPerCAD_weekely.png")

In [7]:
import tweepy
# the below import is for calculating date. Not needed for you but I needed it.
from datetime import date
import shutil, pathlib, os

# take these from developer.twitter.com
from auth import (
    CONSUMER_KEY, 
    CONSUMER_SECRET, 
    ACCESS_KEY, 
    ACCESS_SECRET, 
    BEARER_TOKEN, 
    CLIENT_ID, 
    CLIENT_ID_SECRET
)

#CONSUMER_KEY = 's5H3hUMYvaWmSnGxsCX2a4vet'
#CONSUMER_SECRET = 'YS4zT4ifDM15n97Ifvnd0dF0t1BobVcdDOX2WfwaJycL9vPwO2'

#ACCESS_KEY = '1620163513691025409-vlqYXmjLKcvCybGyQOcIb9YN51oIgi'
#ACCESS_SECRET = 'EC8zWdtonTbigEmDKrbi4ipvlMzlM5Q0ToBG8f7BjyVoU'

#BEARER_TOKEN = 'AAAAAAAAAAAAAAAAAAAAAGSnuwEAAAAAuxh1ICSpy%2BkNHAfmc%2BsWKRtXFkE%3DxVCydOgI7ztOg5IrQYcghRLiqYF7nrk8dIsOo77aAr9SW6BuKT'

#CLIENT_ID = 'UHlBeVNLaU5Za2F6V3JpMkw0NjM6MTpjaQ'
#CLIENT_ID_SECRET = 'v2Ms1N2ggjSVDoP0iwa_ZW2tfHO6BgbTmEy-pKIb4cLdxL9XKb'

# Since I started the event for 30daysofJS on Nov 6 and I wanted to include this is day X in my tweet dynamically, 
# I am setting up the date logic part. This is totally optional.
startdate = date(2023, 11, 6)
today = date.today()
mydays = (today - startdate).days + 1
# End of optional part

# Authenticate to Twitter
auth = tweepy.OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
auth.set_access_token(
    ACCESS_KEY,
    ACCESS_SECRET,
)
# this is the syntax for twitter API 2.0. It uses the client credentials that we created
newapi = tweepy.Client(
    bearer_token=BEARER_TOKEN,
    access_token=ACCESS_KEY,
    access_token_secret=ACCESS_SECRET,
    consumer_key=CONSUMER_KEY,
    consumer_secret=CONSUMER_SECRET,
)

# Create API object using the old twitter APIv1.1
api = tweepy.API(auth)

# adding the tweet content in a multiline string. The {mydays} part is updated dynamically as the number of days from 6th Nov, 2023
sampletweet = f"""test2"""

newapi.create_tweet(text=sampletweet)

# I add the screenshot in the same directory as the code and add only one image. Since it is a screenshot taken using windows Snip, it is in png
#for images in os.listdir(os.path.abspath(os.path.dirname(__file__))):
##    # check if the image ends with png and take the first image that you find
  #  if images.endswith(".png"):
   #     img = images
    #    break

# upload the media using the old api
#media = api.media_upload(os.path.join(os.path.abspath(os.path.dirname(__file__)), img))
# create the tweet using the new api. Mention the image uploaded via the old api
#post_result = newapi.create_tweet(text=sampletweet, media_ids=[media.media_id])
# the following line prints the response that you receive from the API. You can save it or process it in anyway u want. I am just printing it.
#print(post_result)
# get the file extension. This would be png by default but I was experimenting with different images. Will update this if required
#file_extension = pathlib.Path(img).suffix
# Move the image to archives folder and rename it as per the date it was uploaded. I am only adding one post per day so this makes more sense
#shutil.move(
#    os.path.join(os.path.abspath(os.path.dirname(__file__)), img),
##    os.path.join(
 #       os.path.abspath(os.path.dirname(__file__)),
 #       "archives",
  #      today.strftime("%Y%m%d") + file_extension,
#    ),
#)

Forbidden: 403 Forbidden
You are not allowed to create a Tweet with duplicate content.